# Práctica: Lematización para Mejora de Análisis de Sentimiento en Reseñas Multilingües


Implementar un pipeline de lematización que unifique variantes morfológicas y mejore la precisión de un modelo de análisis de sentimiento.


## Fase 1: Diagnóstico de Problemas

In [ ]:

import pandas as pd

df = pd.read_csv("reviews_multilang.csv")
df.head()


In [ ]:

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_counts = vectorizer.fit_transform(df["review_español"])  # suponiendo columna en español
token_freq = pd.DataFrame({'token': vectorizer.get_feature_names_out(), 'freq': X_counts.sum(axis=0).A1})
token_freq.sort_values(by="freq", ascending=False).head(10)


In [ ]:

import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt

nlp = spacy.load("es_core_news_sm")
doc = nlp("Los usuarios reportaron fallas constantes: no funciona, se traba y no responde.")

for token in doc:
    print(token.text, token.pos_, token.lemma_)


In [ ]:

text = " ".join(df["review_español"])
doc = nlp(text)
lemmas = " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
wordcloud = WordCloud(background_color='white').generate(lemmas)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


In [ ]:

variantes = [("fallas", "falla"), ("funciona", "funcionar"), ("trabó", "trabar"), ("responder", "responder"), ("reportaron", "reportar")]
pd.DataFrame(variantes, columns=["Forma Original", "Lema"])


## Fase 2: Implementación del Lematizador

In [ ]:

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lematizar(texto, idioma):
    if idioma == "es":
        doc = spacy.load("es_core_news_sm")(texto)
        return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    elif idioma == "en":
        lemmatizer = WordNetLemmatizer()
        tokens = word_tokenize(texto)
        tagged = pos_tag(tokens)
        return [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in tagged if word.lower() not in stopwords.words("english")]


In [ ]:

print(lematizar("Los dispositivos fallaron constantemente, no funcionan bien.", "es"))


## Fase 3: Optimización y Validación

In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

df["review_lematizado"] = df["review_español"].apply(lambda x: " ".join(lematizar(x, "es")))

vectorizer = TfidfVectorizer()
X_lem = vectorizer.fit_transform(df["review_lematizado"])
X_raw = TfidfVectorizer().fit_transform(df["review_español"])
y = df["sentimiento"]  # asumir que existe esta columna

X_train, X_test, y_train, y_test = train_test_split(X_lem, y, test_size=0.2)
clf = RandomForestClassifier().fit(X_train, y_train)
preds = clf.predict(X_test)
print("F1-score con lematización:", f1_score(y_test, preds, average="macro"))

X_train2, X_test2, _, _ = train_test_split(X_raw, y, test_size=0.2)
clf2 = RandomForestClassifier().fit(X_train2, y_train)
preds2 = clf2.predict(X_test2)
print("F1-score sin lematización:", f1_score(y_test, preds2, average="macro"))


## Fase 4: Evaluación Comparativa

In [ ]:

vocab_size_raw = len(TfidfVectorizer().fit(df["review_español"]).vocabulary_)
vocab_size_lem = len(TfidfVectorizer().fit(df["review_lematizado"]).vocabulary_)
print("Reducción de vocabulario:", round((vocab_size_raw - vocab_size_lem) / vocab_size_raw * 100, 2), "%")


## Entrega Final

La función `lematizar(texto, idioma)` ya está implementada arriba.


### Requiere:
- Comparativo de métricas pre/post lematización.
- Ejemplos de errores corregidos como: "trabó" → "trabar", "funcionó" → "funcionar".
